# 모듈, 데이터 불러오기

In [ ]:
!pip install folium matplotlib mapclassify

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
df_grid = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/unique_grid.csv')
df_grid.head(2)

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구


In [ ]:
crossroad = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/도로시설물/서울시 교차로 관련 정보.csv', encoding = 'cp949')
crossroad.head(2)

,교차로코드,입력일자,교차로명칭,유형코드,연동교차로코드,구코드 (공통);,지번,구경찰서 (공통);,신경찰서 (공통);,작업구분 (공통);,...,관할사업소 (공통);,교차로관리번호,공간데이터,신규정규화ID,이력ID,동코드 (공통);,공사형태 (공통);,지도표출구분,X좌표,Y좌표
0,1439,NaN,오류역입구,1,289.0,530.0,55-1도,330.0,330.0,1,...,104.0,82-0000003357,NaN,1134172.0,3357,10800.0,NaN,N,186235.986159,544102.255410
1,1440,NaN,오류동삼거리,2,264.0,530.0,6-2도,330.0,330.0,1,...,104.0,82-0000003253,NaN,1124532.0,3253,10800.0,NaN,N,185948.049768,543943.862041


# 데이터 정제
- 컬럼중에서 교차로코드, X좌표, Y좌표 컬럼만 추출
- 널값 제외하고 값이 존재하는 데이터만 추출

In [ ]:
crossroad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8126 entries, 0 to 8125
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   교차로코드        8126 non-null   int64  
 1   입력일자         0 non-null      float64
 2   교차로명칭        8123 non-null   object 
 3   유형코드         8126 non-null   int64  
 4   연동교차로코드      6375 non-null   float64
 5   구코드 (공통);    8103 non-null   float64
 6   지번           7236 non-null   object 
 7   구경찰서 (공통);   7870 non-null   float64
 8   신경찰서 (공통);   8100 non-null   float64
 9   작업구분 (공통);   8126 non-null   int64  
 10  표출구분 (공통);   8126 non-null   int64  
 11  고객번호         2 non-null      float64
 12  계약종별         2 non-null      float64
 13  계량기번호        2 non-null      float64
 14  도로구분 (공통);   8048 non-null   float64
 15  관할사업소 (공통);  8082 non-null   float64
 16  교차로관리번호      8126 non-null   object 
 17  공간데이터        0 non-null      float64
 18  신규정규화ID      5222 non-null   float64
 19  이력ID  

In [ ]:
crossroad = crossroad.loc[ :, ['교차로코드', 'X좌표' , 'Y좌표']]
clean_crossroad = crossroad[crossroad.X좌표.isnull() == False ]

In [ ]:
clean_crossroad.info()
clean_crossroad.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8103 entries, 0 to 8124
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   교차로코드   8103 non-null   int64  
 1   X좌표     8103 non-null   float64
 2   Y좌표     8103 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 253.2 KB


,교차로코드,X좌표,Y좌표
0,1439,186235.986159,544102.25541


# 교차로 데이터 프레임
```
기존에 .csv 파일이었기 때문에 'geometry'열이 존재하지 않는다.

따라서 geodataframe을 구성할 때 x좌표와 y좌표를 이용하여 'geometry' 컬럼을 만들어준다. 

그 후 to_crs() 함수를 이용해 좌표계를 5179 좌표계로 변경한다.

좌표계 설정 이유 : 좌표계의 종류가 많기 때문에 어떤 종류의 좌표계인지 명시해주어야 함

```


In [ ]:
geo_crossroad = gpd.GeoDataFrame(
    clean_crossroad, geometry=gpd.points_from_xy(clean_crossroad['X좌표'], clean_crossroad['Y좌표'])
)
geo_crossroad.set_crs(epsg = 5186, inplace = True)
geo_crossroad = geo_crossroad.to_crs(epsg=5179)

In [ ]:
geo_crossroad.explore(tiles='http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}', attr='Google')

Output hidden; open in https://colab.research.google.com to view.

## 격자지도 데이터 프레임
```
기존에 .shp을 로드해서 csv로 저장했기 때문에 'geometry'열이 존재한다.

그러나 csv로 저장하면서 기존에 'geometry'열의 데이터 타입이 문자열로 변경되었다.

따라서 wkt.loads를 이용해서 문자열 타입을 폴리곤 타입으로 바꾸고, geodataframe을 구성할 때 좌표계 정보(epsg:5179)를 주었다.

```

In [ ]:
from shapely import wkt

df_grid['geometry'] = df_grid['geometry'].apply(wkt.loads)
geo_grids = gpd.GeoDataFrame(df_grid, crs='epsg:5179')

## geopandas 의 sjoin을 사용

- 정확히는 모르지만 공간에 따라 데이터를 join 하는 것 같음
- pandas.join과 유사 ( 다른 점: pandas의 join은 데이터가 같은 것끼리 엮지만 geopandas의 sjoin은 공간이 겹치면 엮는다 )

In [ ]:
join_geo_df = gpd.sjoin(geo_crossroad, geo_grids, how='left')
join_geo_df.shape

(8103, 9)

In [ ]:
# 피벗테이블 이용해서 격자지도별로 교차로 카운트
crossroad_by_grid = join_geo_df.pivot_table(index='gid', values='교차로코드', aggfunc='count')
crossroad_by_grid.head(5)

,교차로코드
gid,
다사38a50a,1
다사38a51b,2
다사38a52b,4
다사38a53a,3
다사38a53b,4


In [ ]:
crossroad_by_grid.sort_values(by='교차로코드', ascending=False).head(5)

,교차로코드
gid,
다사51b51a,20
다사44b47b,15
다사46a44b,15
다사62b57a,14
다사39a53a,14


In [ ]:
crossroad_by_grid.to_csv('crossroad_by_grid')